In [129]:
# Import the dependencies
import pandas as pd
import numpy as np
import datetime

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

# ORM Stuff
from sqlalchemy import create_engine, inspect, text, func


In [130]:
df = pd.read_csv("Olympic_Swimming_Results_1912to2020.csv")
print(df.shape)
df.head()

(4359, 10)


,Location,Year,Distance (in meters),Stroke,Relay?,Gender,Team,Athlete,Results,Rank
0,Tokyo,2020,100m,Backstroke,0,Men,ROC,Evgeny Rylov,51.98,1
1,Tokyo,2020,100m,Backstroke,0,Men,ROC,Kliment Kolesnikov,52,2
2,Tokyo,2020,100m,Backstroke,0,Men,USA,Ryan Murphy,52.19,3
3,Tokyo,2020,100m,Backstroke,0,Men,ITA,Thomas Ceccon,52.3,4
4,Tokyo,2020,100m,Backstroke,0,Men,CHN,Jiayu Xu,52.51,4


In [131]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4359 entries, 0 to 4358
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Location              4359 non-null   object
 1   Year                  4359 non-null   int64 
 2   Distance (in meters)  4359 non-null   object
 3   Stroke                4359 non-null   object
 4   Relay?                4359 non-null   int64 
 5   Gender                4359 non-null   object
 6   Team                  4359 non-null   object
 7   Athlete               4345 non-null   object
 8   Results               4331 non-null   object
 9   Rank                  4359 non-null   int64 
dtypes: int64(3), object(7)
memory usage: 340.7+ KB


In [132]:
## Drop unwanted 'Relay' column 
df = df.drop(columns=['Relay?'])  # Drop the column
df = df.dropna()  
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4317 entries, 0 to 4358
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Location              4317 non-null   object
 1   Year                  4317 non-null   int64 
 2   Distance (in meters)  4317 non-null   object
 3   Stroke                4317 non-null   object
 4   Gender                4317 non-null   object
 5   Team                  4317 non-null   object
 6   Athlete               4317 non-null   object
 7   Results               4317 non-null   object
 8   Rank                  4317 non-null   int64 
dtypes: int64(2), object(7)
memory usage: 337.3+ KB


In [133]:
engine = create_engine("sqlite:///Olympic_Swimming_Results.sqlite")

In [134]:
# Random sample for SPEED
df2 = df.sample(4000)
df2.head()

,Location,Year,Distance (in meters),Stroke,Gender,Team,Athlete,Results,Rank
1814,Barcelona,1992,100m,Freestyle,Men,GER,Christian Tröger,49.840,4
2066,Seoul,1988,100m,Freestyle,Women,FRA,Catherine Plewinski,55.490,3
1832,Barcelona,1992,200m,Backstroke,Men,ESP,Martin Lopez Zubero,00:01:58.470000,1
4335,Stockholm,1912,400m,Breaststroke,Men,GER,Walter Bathe,00:06:29.600000,1
182,Tokyo,2020,200m,Freestyle,Women,ITA,Federica Pellegrini,1:55.91,4


In [135]:
df2.to_sql(name="Olympic_Swimming_Results", con=engine, index=False, if_exists="append", method="multi")


4000

In [136]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)

# Collect the names of tables within the database
tables = inspector.get_table_names()

# Using the inspector to print the column names within the 'dow' table and its types
for table in tables:
    print(table)
    print("--------")
    columns = inspector.get_columns(table)
    for column in columns:
        print(column["name"], column["type"])

    print()

Olympic_Swimming_Results
--------
Location TEXT
Year BIGINT
Distance (in meters) TEXT
Stroke TEXT
Gender TEXT
Team TEXT
Athlete TEXT
Results TEXT
Rank BIGINT



In [137]:
conn = engine.connect() # Raw SQL/Pandas

In [138]:
 # Raw SQL
query = text("""SELECT COUNT(*) AS total_records FROM Olympic_Swimming_Results;""")
df2 = pd.read_sql(query, con=conn)
df2.head(10)

,total_records
0,17318
